In [1]:
import glob
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [3]:
LEARNING_RATE = 0.003
DECAY_STEPS = 1000
DECAY_RATE = 2
EPSILON = 1e-06
BEST_PATH = './models/Baseline_conv.h5'

In [4]:
train_files = glob.glob('./data/train/*.npy')
train_files = shuffle(train_files, random_state=3101)
len(train_files)

62735

In [5]:
def trainGenerator():
    for file in train_files:
        dataset = np.load(file)
        target= dataset[:,:,-1].reshape(120,120,1)
        remove_minus = np.where(target < 0, 0, target)
        feature = dataset[:,:,:4]

        yield (feature, remove_minus)

In [6]:
NUM_TRAIN = int(len(train_files)*.7)

full_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([120,120,4]),tf.TensorShape([120,120,1])))

train_dataset = full_dataset.take(NUM_TRAIN)
train_dataset = train_dataset.batch(64).prefetch(1)
val_dataset = full_dataset.skip(NUM_TRAIN)
val_dataset = val_dataset.batch(64).prefetch(1)

In [7]:
def base_model(input_layer, start_neurons):
    
    conv1 = layers.Conv2D(start_neurons * 1, (3, 3), activation=tf.nn.leaky_relu, padding="same")(input_layer)
    pool1 = layers.BatchNormalization()(conv1)
    pool1 = layers.MaxPooling2D((2, 2))(pool1)

    conv2 = layers.Conv2D(start_neurons * 2, (3, 3), activation=tf.nn.leaky_relu, padding="same")(pool1)
    pool2 = layers.BatchNormalization()(conv2)
    pool2 = layers.MaxPooling2D((2, 2))(pool2)

    convm = layers.Conv2D(start_neurons * 4, (3, 3), activation=tf.nn.leaky_relu, padding="same")(pool2)

    deconv2 = layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = layers.concatenate([deconv2, conv2])
    uconv2 = layers.Conv2D(start_neurons * 2, (3, 3), activation=tf.nn.leaky_relu, padding="same")(uconv2)
    uconv2 = layers.BatchNormalization()(uconv2)

    deconv1 = layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = layers.concatenate([deconv1, conv1])
    uconv1 = layers.Conv2D(start_neurons * 1, (3, 3), activation=tf.nn.leaky_relu, padding="same")(uconv1)
    uconv1 = layers.BatchNormalization()(uconv1)
    output_layer = layers.Conv2D(1, (1,1), padding="same", activation=tf.nn.leaky_relu)(uconv1)
    
    return output_layer

input_layer = Input((120, 120, 4))
output_layer = base_model(input_layer,64)

In [8]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_PATH,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,  patience=20)
]

In [9]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [10]:
lr_schedule = optimizers.schedules.InverseTimeDecay(LEARNING_RATE, DECAY_STEPS, DECAY_RATE, staircase=True)

In [11]:
model = Model(input_layer, output_layer)
opt = tf.optimizers.Adam(learning_rate=lr_schedule, epsilon=EPSILON)
model.compile(loss='mae', optimizer=opt)
model.fit(train_dataset, epochs = 200,
          callbacks = callbacks, validation_data=val_dataset,
          verbose=1)

Epoch 1/200
    687/Unknown - 78s 113ms/step - loss: 4.6160
Epoch 00001: val_loss improved from inf to 3.63566, saving model to ./models/Baseline_conv.h5
687/687 [==============================] - 127s 185ms/step - loss: 4.6160 - val_loss: 3.6357
Epoch 2/200
686/687 [============================>.] - ETA: 0s - loss: 3.6869
Epoch 00002: val_loss improved from 3.63566 to 3.26491, saving model to ./models/Baseline_conv.h5
687/687 [==============================] - 127s 185ms/step - loss: 3.6875 - val_loss: 3.2649
Epoch 3/200
686/687 [============================>.] - ETA: 0s - loss: 3.5659
Epoch 00003: val_loss improved from 3.26491 to 3.25149, saving model to ./models/Baseline_conv.h5
687/687 [==============================] - 127s 185ms/step - loss: 3.5665 - val_loss: 3.2515
Epoch 4/200
686/687 [============================>.] - ETA: 0s - loss: 3.4970
Epoch 00004: val_loss improved from 3.25149 to 3.19515, saving model to ./models/Baseline_conv.h5
687/687 [==============================

In [12]:
test_path = './data/test'
test_files = sorted(glob.glob(test_path + '/*.npy'))

X_test = []

for file in tqdm(test_files, desc = 'test'):
    data = np.load(file)
    X_test.append(data)

X_test = np.array(X_test)

test: 100%|██████████| 2674/2674 [00:00<00:00, 3171.22it/s]


In [13]:
X_test.shape

(2674, 120, 120, 4)

In [14]:
pred = model.predict(X_test)

In [15]:
submission = pd.read_csv('./data/sample_submission.csv')

In [16]:
submission.iloc[:,1:] = pred.reshape(-1, 14400).astype(int)
submission.to_csv('./results/Dacon_baseline.csv', index = False)